In [1]:
!pip install pandas numpy

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

cities = ["Delhi", "Mumbai", "Kolkata", "Chennai", "Hyderabad", "Bengaluru"]

lat = [28.7041, 19.0760, 22.5726, 13.0827, 17.3850, 12.9716]
lon = [77.1025, 72.8777, 88.3639, 80.2707, 78.4867, 77.5946]

base_df = pd.DataFrame({
    "city": cities,
    "lat": lat,
    "lon": lon
})

def generate_realtime_data():
    realtime_list = []
    current_time = datetime.now()
    for i in range(48):
        ts = current_time + timedelta(hours=i)
        for city, la, lo in zip(cities, lat, lon):
            aqi = np.random.randint(120, 400)
            heat = np.random.uniform(26, 42)
            rain = np.random.uniform(0, 120)
            wind = np.random.uniform(5, 28)
            seismic = np.random.uniform(0.1, 6.5)
            realtime_list.append([city, la, lo, ts, aqi, heat, rain, wind, seismic])
    df = pd.DataFrame(realtime_list, columns=[
        "city","lat","lon","timestamp",
        "AQI","heat_index","rainfall","wind_speed","seismic_activity"
    ])
    return df

df = generate_realtime_data()
df.to_csv("realtime_hazard_data.csv", index=False)

def hazard_score(row):
    score = (
        row["AQI"] * 0.30 +
        row["heat_index"] * 0.20 +
        row["rainfall"] * 0.20 +
        row["wind_speed"] * 0.10 +
        row["seismic_activity"] * 0.20
    )
    return round(score,2)

df["hazard_score"] = df.apply(hazard_score, axis=1)

def classify_alert(score):
    if score >= 90:
        return "RED ALERT"
    elif score >= 70:
        return "ORANGE ALERT"
    elif score >= 50:
        return "YELLOW ALERT"
    else:
        return "NORMAL"

df["alert_level"] = df["hazard_score"].apply(classify_alert)

df.to_csv("disaster_alerts_48hrs.csv", index=False)

alerts = df[df["alert_level"] != "NORMAL"]

with open("alert_report.txt","w") as f:
    f.write("Early Warning Disaster Alert Report\n")
    f.write("----------------------------------\n\n")
    for _, row in alerts.iterrows():
        f.write(f"{row['timestamp']} - {row['city']} - {row['alert_level']}\n")
        f.write(f"AQI: {row['AQI']}, Heat: {row['heat_index']:.1f}, Rain: {row['rainfall']:.1f}, Wind: {row['wind_speed']:.1f}, Seismic: {row['seismic_activity']:.1f}\n\n")

print("Saved: realtime_hazard_data.csv")
print("Saved: disaster_alerts_48hrs.csv")
print("Saved: alert_report.txt")

Saved: realtime_hazard_data.csv
Saved: disaster_alerts_48hrs.csv
Saved: alert_report.txt


In [2]:
!zip -r branch7_outputs.zip realtime_hazard_data.csv disaster_alerts_48hrs.csv alert_report.txt

  adding: realtime_hazard_data.csv (deflated 64%)
  adding: disaster_alerts_48hrs.csv (deflated 65%)
  adding: alert_report.txt (deflated 85%)
